## Imports

In [ ]:
!pip install PyPDF2 --quiet
!pip install keybert --quiet

In [ ]:
import torch

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import json

In [ ]:
# for text preprocessing
import re
import spacy

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string

# import vectorizers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
from nltk.tokenize import RegexpTokenizer

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# import wordnet
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from gensim.models import Phrases
from gensim.corpora import Dictionary

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
from gensim.models import Phrases

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
from gensim.models import LdaModel, LdaMulticore

In [ ]:
import re

In [ ]:
from collections import defaultdict
import PyPDF2

In [ ]:
# !pip install flair


In [ ]:
from keybert import KeyBERT
from keybert.backend import BaseEmbedder
# from flair.embeddings import TransformerDocumentEmbeddings

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

# tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
# query_model = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import io

In [ ]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [ ]:
import faiss

In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
import re

In [ ]:
!pip install langchain
!pip install ctransformers
!pip install ctransformers[cuda]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.9/378.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 21.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install groq
!pip install langchain-groq

In [ ]:
from langchain_groq import ChatGroq

## Paragraph

In [ ]:
# pattern = r'fig(ure)?.?.?[0-9]+.[0-9]+'
# re.search( string="This is a figure 1.5 and it works", flags=re.IGNORECASE, pattern=pattern)

In [ ]:
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

MODEL_STRING = "facebook/dpr-ctx_encoder-single-nq-base"

In [ ]:
class Paragraph:
    def __init__(self, book_id:int, id: int, text: str, embeddings: np.array, tagged_images: list=None):
        self.text = text
        self.embeddings = embeddings
        self.id = id
        self.book_id = book_id
    def get_json(self):
        dc = {
            "text": self.text,
            "id": self.id,
            "embeddings": self.embeddings
        }
        return json.dumps(dc)

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer

In [ ]:
class EncoderModels():
    def __init__(self):
        ctx_model_string = "facebook/dpr-ctx_encoder-single-nq-base"
        query_model_string = "facebook/dpr-question_encoder-single-nq-base"

        DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
        self.device = DEVICE

        self.ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained(ctx_model_string)
        self.ctx_model = DPRContextEncoder.from_pretrained(ctx_model_string)
        self.ctx_model = self.ctx_model.to(DEVICE)

        self.query_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(query_model_string)
        self.query_model = DPRQuestionEncoder.from_pretrained(query_model_string)
        self.query_model = self.query_model.to(DEVICE)

    def encode_context(self, paragraph: list[str],):
        num_paragraphs = len(paragraph)
        cloned_doc_embeddings = []
        batch_size = 32
        # print(paragraph)
        for itr in range(0, num_paragraphs, batch_size):
            s = itr
            e = min(itr+batch_size, num_paragraphs)
            tokenized = self.ctx_tokenizer(paragraph[s : e],
                                       padding=True, truncation=True,
                                       return_tensors="pt",
                                       return_attention_mask=True,
                                       add_special_tokens=True,
                                       max_length=512,
                                       pad_to_max_length=True
                                       ).to(self.device)
            input_ids = tokenized['input_ids']
            print(input_ids.shape)
            attention_mask = tokenized['attention_mask']
            print(attention_mask)
            with torch.no_grad():
                doc_embeddings = self.ctx_model(input_ids, attention_mask=attention_mask).pooler_output

            cloned_doc_embeddings.append(torch.clone(doc_embeddings).cpu().numpy())

            ##clean up
            torch.cuda.empty_cache()
            del input_ids

        return np.vstack(cloned_doc_embeddings)
    def encode_query(self, query):

        tokenized = self.query_tokenizer(query,
                                    padding=True, truncation=True,
                                    return_tensors="pt",
                                    return_attention_mask=True,
                                    add_special_tokens=True,
                                    max_length=512,
                                    pad_to_max_length=True
                                    ).to(self.device)
        input_ids = tokenized['input_ids']
        attention_mask = tokenized['attention_mask']
        with torch.no_grad():
            doc_embeddings = self.query_model(input_ids, attention_mask=attention_mask).pooler_output
        embedding = torch.clone(doc_embeddings).cpu().numpy()

        ##clean up
        torch.cuda.empty_cache()
        del input_ids

        return embedding





In [ ]:
class EncoderModel():
    def __init__(self, model_string: str = MODEL_STRING, device: str= DEVICE):
        super().__init__()
        self.device = DEVICE
        self.model_string = model_string

        self.tokenizer = AutoTokenizer.from_pretrained(model_string)
        self.model = AutoModel.from_pretrained(model_string, resume_download=True)
        self.model = self.model.to(DEVICE)
        # self.tokenizer = self.tokenizer.to(self.device)


    def encode(self, paragraph: list[str],):
        num_paragraphs = len(paragraph)
        cloned_doc_embeddings = []
        batch_size = 32
        print(paragraph)
        for itr in range(0, num_paragraphs, batch_size):
            s = itr
            e = min(itr+batch_size, num_paragraphs)
            tokenized = self.tokenizer(paragraph[s : e],
                                       padding=True, truncation=True,
                                       return_tensors="pt",
                                       return_attention_mask=True,
                                       add_special_tokens=True,
                                       max_length=512,
                                       pad_to_max_length=True
                                       ).to(self.device)
            input_ids = tokenized['input_ids']
            print(input_ids.shape)
            attention_mask = tokenized['attention_mask']
            print(attention_mask)
            with torch.no_grad():
                doc_embeddings = self.model(input_ids, attention_mask=attention_mask).pooler_output

            cloned_doc_embeddings.append(torch.clone(doc_embeddings).cpu().numpy())

            ##clean up
            torch.cuda.empty_cache()
            del input_ids

        return np.vstack(cloned_doc_embeddings)


In [ ]:
# encoder = Encoder_Model()

In [ ]:
# emb = encoder.encode(["Its been a long night","tomowwor we win this game by hige margin"])

In [ ]:
# emb.shape

## Pdf Parser

In [ ]:
class PdfParagraphExtractor:
    def __init__(self, book_path: str) -> None:
        self.book_path = book_path

        self.reader = PyPDF2.PdfReader(book_path)

        self.paragraphs = []
        self.paragraph_count = 0

        self.extract_paragraphs()
    def extract_paragraphs(self):
        for page in self.reader.pages:
            text = page.extract_text()
            paragraphs = text.split('.\n')
            for paragraph in paragraphs:
                paragraph = paragraph.replace('\n', '').replace('\t', '')
                if(paragraph!='' and paragraph!=None and len(paragraph.split())>0):
                    self.paragraphs.append(paragraph)
        self.paragraph_count = len(self.paragraphs)

    def get_paragraphs(self):
        return self.paragraphs
    def get_paragraph_count(self):
        return self.paragraph_count
    def get_paragraph(self, index: int):
        return self.paragraphs[index]




## Faiss

In [ ]:
class Faiss:
    def __init__(self,doc_embeddings,model,top_k=15,dimension=768):
        # doc embeddings is an array of vectors of shape (1,768)
        self.doc_embeddings = np.vstack(doc_embeddings)
        self.top_k=top_k
        self.dimension=dimension
        res = faiss.StandardGpuResources()
        flat_config = faiss.GpuIndexFlatConfig()

        faiss.normalize_L2(self.doc_embeddings)
        self.index=faiss.GpuIndexFlatL2(res,self.dimension,flat_config)

        self.query_model=model
        self.create_index()
    def create_index(self):
        # Add your vector data to the index
        self.index.add(self.doc_embeddings)
        return self.index.ntotal
    def serve_query(self,question):
        query_embedding = self.query_model.encode_query(question)

        faiss.normalize_L2(query_embedding)

        D, I = self.index.search(query_embedding, self.top_k)
        return D,I


## Hypergraph

In [ ]:
class Hypergraph():
    # all the hypergraph related mapping and its index
    def __init__(self, name, key_to_docids, key_to_embeddings, key_to_keyid, keyid_to_key, encoder_models):
        self.name = name
        self.key_to_docids = key_to_docids
        self.key_to_embeddings = key_to_embeddings
        self.key_to_keyid = key_to_keyid
        self.keyid_to_key = keyid_to_key

        self.vector_db = Faiss(list(self.key_to_embeddings.values()), encoder_models)

    def query_hypergraph(self, question):
        D, I = self.vector_db.serve_query(question)
        key_ids = I[0]
        ans_arr = []
        for key_id in key_ids:
            key = self.keyid_to_key[key_id]
            docids = self.key_to_docids[key]
            ans_arr.extend(docids)
        return ans_arr




## Pdf wrapper class

In [ ]:
class Models():
    def __init__(self):
        self.encoder_models = EncoderModels()

In [ ]:
class Book():
    def __init__(self, book_paths, models):
        self.book_paths = book_paths
        self.models = models

        #some global identifiers
        self.global_paragraphs = None
        self.global_doc_embeddings = None
        self.pobj = None
        self.num_paragraphs = 0

        #read book
        self.books = [PdfParagraphExtractor(book_path) for book_path in book_paths]
        print(self.books[0].paragraphs)
        self.bookid_to_pid = defaultdict(list)
        self.pid_to_bookid = defaultdict(int)
        self.set_global_info()

        self.normal_hg = self.generate_normal_hg()

    def get_paragraphs(self):
        return self.global_paragraphs
    def set_global_info(self):
        all_ps = []
        all_objs = []
        all_doc_embeddings = []

        id = 0
        for bookid, book in enumerate(self.books):
            doc_embeddings_by_book = self.compute_doc_embeddings(book.paragraphs)

            for pid, paragraph in enumerate(book.paragraphs):
                all_ps.append(paragraph)
                all_doc_embeddings.append(doc_embeddings_by_book[pid])

                self.bookid_to_pid[bookid].append(id)
                self.pid_to_bookid[id] = bookid
                id+=1

        self.num_paragraphs = len(all_ps)
        self.global_paragraphs = all_ps
        self.global_doc_embeddings = all_doc_embeddings

        for id in range(self.num_paragraphs):
            all_objs.append(Paragraph(self.pid_to_bookid[id], id, all_ps[id], all_doc_embeddings[id]))
        self.pobj = all_objs

        return

    def compute_doc_embeddings(self, para_list):
        doc_embeddings = self.models.encoder_models.encode_context(para_list)
        r, c = doc_embeddings.shape
        doc_embeddings = [doc_emb[:].reshape(1,-1) for doc_emb in doc_embeddings]
        return doc_embeddings

    def get_docs_from_docids(self, docids):
        ans = []
        for docid in docids:
            ans.append(self.global_paragraphs[docid])
        return " ".join(ans)

    def generate_normal_hg(self):
        n = len(self.global_paragraphs)
        key_to_docids = {}
        key_to_keyid = {}
        keyid_to_key = {}
        key_to_embeddings = {}
        for i in range(n):
            key_to_docids[i] = [i]
            key_to_keyid[i] = i
            keyid_to_key[i] = i
            key_to_embeddings[i] = self.global_doc_embeddings[i]

        normal = Hypergraph('normal', key_to_docids, key_to_embeddings, key_to_keyid, keyid_to_key, self.models.encoder_models)
        return normal

    def query_hypergraph(self, question):
        doc_ids = self.normal_hg.query_hypergraph(question)
        context = self.get_docs_from_docids(doc_ids)
        return context



In [ ]:
groq_api_key="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"

# st.title("Objectbox VectorstoreDB With Llama3 Demo")

llm=ChatGroq(groq_api_key=groq_api_key,
             model_name="Llama3-8b-8192")

In [ ]:
def generate_answer(question: str, bookObj: Book):
    extracted_text = bookObj.query_hypergraph(question)

    prompt = f'Given the context: {extracted_text} /n Answer the question {question}'
    print(prompt)

    response = llm.invoke(prompt)

    return response.content

In [ ]:
book_paths = ['/content/test2.pdf', '/content/test.pdf']

In [ ]:
models = Models()

NameError: name 'Models' is not defined

In [ ]:
books = Book(book_paths, models)

In [ ]:
question = 'What is scope of work for MPDA?'

In [ ]:
generate_answer(question, books)

## Streamlit

In [ ]:
!pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.9 MB/s eta 0:00:00


In [ ]:
!npm install localtunnel


added 22 packages, and audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues, run:
  npm audit fix

Run `npm audit` for details.


In [ ]:
# %%writefile app.py

# import streamlit as st

# st.write('Hello, *World!* :sunglasses:')

Writing app.py


In [ ]:
def PRINT(ac):
    print(ac)
    return ac

In [ ]:
%%writefile app.py

import streamlit as st
import io
import PyPDF2
from langchain_groq import ChatGroq

st.title("Echo Bot")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("Say Something"):
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    response = f"Echo: {prompt}"
    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        st.markdown(response)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})


uploaded_file = st.file_uploader("Choose a file", 'pdf')
if uploaded_file is not None:
    # To read file as bytes:
    bytes_data = uploaded_file.getvalue()
    # st.write(bytes_data)

    bytes_stream_data = io.BytesIO(bytes_data)
    pdfObj = PyPDF2.PdfReader(bytes_stream_data)
    # st.write(pdfObj)


Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

your url is: https://young-tips-march.loca.lt
